# Book Summariser (WIP)

I am part of a book club, and I wanted the ability to go back and review books I've already read and question the text. 
Example use would be to find out:
- What themes were prominent in the book? 
- What would be some open ended questions that can drive discussion about aspects about this book? 
- Get a description of all the main characters in the book.


I am having trouble uploading a file to the genai api. Possible things to try:
- Try running this in Colab
- Try using the Vertex API if that doesn't work. 
- Find out how to get a similar experience of Google AI studios. 

In [1]:
import os
import google.generativeai as genai
from dotenv import load_dotenv, find_dotenv
import zipfile
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
import mimetypes
import time

load_dotenv(find_dotenv())

True

In [2]:
genai.configure(api_key=os.environ.get('GEMINI_API_KEY'))

In [3]:

def book_text(file_path):
    """reads the book text into memory as a string to feed a prompt.

    Args:
      file_path: path to the book txt file. 

    Returns:
      String of the book text.
    """
    with open(file_path, "r") as f:
        text = f.read()
    
    return text



def summarize_text(book_text):
    """Summarizes a text file using the Gemini API.

    Args:
      book_text: the book text as a string.

    Returns:
      A summary of the text.
    """
    model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    response = model.generate_content(f"Please provide a concise summary of the following Book:\n\n{book_text}")
    # response = genai.generate_content(
    #     model="gemini-pro",
    #     prompt=f"Please provide a concise summary of the following Book:\n\n{book_text}",
    # )
    return response.result


def gemini_prompt(book_text, my_prompt):
    """Allows you to ask questions about the following book text.

    Args:
      book_text: the book text as a string.
      my_prompt: the prompt.

    Returns:
      answer from gemini api.
    """        
    model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    response = model.generate_content(f"Please provide a concise summary of the following Book:\n\n{book_text}")
    # response = genai.generate_text(
    #     model="gemini-pro",
    #     prompt=f"Please provide a concise summary of the following Book:\n\n{book_text}",
    # )
    return response.result


In [4]:
# # Determine MIME type
# mime_type, _ = mimetypes.guess_type("book_output copy.txt")
# if mime_type is None:
#     mime_type = "application/octet-stream"  # Fallback to generic type

# # Upload the file
# with open(file_path, "r") as f:
#     file_contents = f.read()

# import pathlib

# media = pathlib.Path(__file__).parents[1] / "third_party"

# file_hash = genai.upload_file(media / "book_output copy.txt")

# print(file_hash)

In [ ]:
# # Example usage
# file_path = "book_output copy.txt"  # Replace with your file path
# query = "Return a summary of the following book"
# response = upload_and_query_file(file_path, query)
# print(response)


# -------------------------------------

# book_txt = book_text("book_output.txt")
# summarize_text(book_txt)

: 

In [ ]:


def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

def wait_for_files_active(files):
  """Waits for the given files to be active.

  Some files uploaded to the Gemini API need to be processed before they can be
  used as prompt inputs. The status can be seen by querying the file's "state"
  field.

  This implementation uses a simple blocking polling loop. Production code
  should probably employ a more sophisticated approach.
  """
  print("Waiting for file processing...")
  for name in (file.name for file in files):
    file = genai.get_file(name)
    while file.state.name == "PROCESSING":
      print(".", end="", flush=True)
      time.sleep(10)
      file = genai.get_file(name)
    if file.state.name != "ACTIVE":
      raise Exception(f"File {file.name} failed to process")
  print("...all files ready")
  print()

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-pro",
  generation_config=generation_config,
)

# You may need to update the file paths
files = [
  upload_to_gemini("book_output.txt", mime_type="text/plain"),
]

# Some files have a processing delay. Wait for them to be ready.
wait_for_files_active(files)

chat_session = model.start_chat()

response = chat_session.send_message("List all the characters of the uploaded book, from most important to least important")

print(response.text)



In [ ]:
# def epub_to_text(epub_path):
#     """Converts an EPUB file to a plain text file.

#     Args:
#       epub_path: Path to the EPUB file.

#     Returns:
#       The extracted plain text.
#     """

#     text = ""
#     with zipfile.ZipFile(epub_path, 'r') as zf:
#         for filename in zf.namelist():
#             if filename.endswith('.html') or filename.endswith('.xhtml'):
#                 with zf.open(filename) as f:
#                     content = f.read().decode('utf-8')
#                     soup = BeautifulSoup(content, 'html.parser')
#                     text += soup.get_text()
#     return text


In [ ]:
# after I convert the epub to text I will go in and edit the text to make it easier to read when I feed it into gemini.
# book_txt = epub_to_text(r"C:\Users\harpo\Desktop\A Darker Shade of Magic.epub")
# with open("book_output.txt", "w") as f:
#     f.write(book_txt)